In [1]:
import Pkg

Pkg.activate("../Project.toml")

using CUDA
using Statistics
using Random
using Test
import Base: length, size, iterate, eltype, IteratorSize, IteratorEltype, haslength, @propagate_inbounds, repeat, rand, tail
import .Iterators: cycle, Cycle, take
using Plots; default(fmt=:png,ls=:auto)

import Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!

using MLDatasets: MNIST

 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`
┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1278


In [2]:
include("../nn.jl")

import .NN

function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end

train! (generic function with 2 methods)

In [ ]:

xtrn, ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10;
xtst, ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10;

In [4]:

include("../utils/parse_config.jl")
include("../models.jl")
Random.seed!(1)

darknet = Darknet("../mnist.cfg"; verbose=false);


model = darknet

xtrn, ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10;
xtst, ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10;
xtrn = reshape(xtrn, (28, 28, 1, :))
xtrn = xtrn[:,:,:,1:6]
xtrn = unpool(xtrn; window=15)
xtrn = xtrn[3:418, 3:418, :, :]
xtst = reshape(xtst, (28, 28, 1, :))
xtst = xtst[:,:,:,1:4]
xtst = unpool(xtst; window=15)
xtst = xtst[3:418, 3:418, :, :]
# ytrn = rand(1:85, (1, size(yolo_re)[2:end]...))


dtrn = minibatch(xtrn, ytrn[1:6], 2; xsize = (416,416,1,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(xtst, ytst[1:4], 2; xsize = (416,416,1,:), xtype=Knet.atype());


iters, trnloss, trnacc, tstloss, tstacc = train!(
    model, dtrn, dtst; 
    period=1, iters=52, lr=1e-5, optimizer=sgd!);

# @time train!(
#     model, dtrn, dtst; 
#     period=1, iters=10, lr=0.15, optimizer=sgd!);


plot(iters, trnloss, label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, tstloss, label="test"))

plot(iters, round.(1 .- trnacc, digits=2), label="train", xlabel="Iterations", ylabel="Misclassification error")
display(plot!(iters, round.(1 .- tstacc, digits=2), label="test"))

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 2 and 6")

In [60]:

include("utils/parse_config.jl")
include("models.jl")

darknet = Darknet("mnist.cfg"; verbose=false);

x,y = first(dtrn);

yolo_out = darknet(x);

4-element Array{Int64,1}:
  2
  1
 10
  4

In [61]:
darknet(x, y)

4.44875f0

In [6]:


xtrn, ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10;
xtst, ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10;
dtrn = minibatch(xtrn, ytrn, 100; xsize = (28,28,1,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(xtst, ytst, 100; xsize = (28,28,1,:), xtype=Knet.atype());
x,y = first(dtrn)
size(x)
model(reshape(xtrn[:,:, 1:20], (28, 28, 1, :)), ytrn[1:20])

Array{Float32,4}Array{Int64,1}


In [7]:
xtrn, ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10;
xtst, ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10;
xtrn = reshape(xtrn, (28, 28, 1, :))
xtrn = xtrn[:,:,:,1:20]
xtrn = unpool(xtrn; window=15)
xtrn = xtrn[3:418, 3:418, :, :]
xtst = reshape(xtst, (28, 28, 1, :))
xtst = xtst[:,:,:,1:8]
xtst = unpool(xtst; window=15)
xtst = xtst[3:418, 3:418, :, :]
dtrn = minibatch(xtrn, ytrn[1:20], 4; xsize = (416,416,1,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(xtst, ytst[1:8], 4; xsize = (416,416,1,:), xtype=Knet.atype());


In [8]:
include("utils/parse_config.jl")
include("models.jl")

darknet = Darknet("mnist.cfg"; verbose=false);



x, y = first(dtst)
yolo_out = darknet(x);

(416, 416)


In [ ]:
Pkg.add("JLD2")

In [36]:
include("nn.jl")

import .NN

function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
    
        push!(train_acc, model(train_data; accuracy=true))
        push!(test_acc, model(test_data; accuracy=true))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss)
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end

train! (generic function with 2 methods)